In [ ]:
# -*- coding: utf-8 -*-
from utils.agent import *
from utils.dataset import *

import random
from IPython.display import clear_output
import sys
import traceback
import sys
import os
import tqdm.notebook as tq
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


try:
    if "google.colab" in str(get_ipython()):
        from google.colab import drive

        drive.mount("/content/gdrive")
        LOAD = True
        SAVE_MODEL_PATH = "/content/gdrive/MyDrive/models/" + "q_network"
    else:
        LOAD = False
        SAVE_MODEL_PATH = "./models/q_network"
except NameError:
    LOAD = False
    SAVE_MODEL_PATH = "./models/q_network"

batch_size = 32
PATH = "./datasets/"
arch_type = "resnet50"


train_loader2012, val_loader2012 = read_voc_dataset(download=LOAD, year="2012")
train_loader2007, val_loader2007 = read_voc_dataset(download=LOAD, year="2007")

### Séparation des différents éléments du jeu de données selon la classe.


In [ ]:
classes = [
    "cat",
    "dog",
    "bird",
    "motorbike",
    "diningtable",
    "train",
    "tvmonitor",
    "bus",
    "horse",
    "car",
    "pottedplant",
    "person",
    "chair",
    "boat",
    "bottle",
    "bicycle",
    "aeroplane",
    "cow",
    "sheep",
    "sofa",
]


agents_per_class = {}
datasets_per_class = sort_class_extract([train_loader2007, train_loader2012])
datasets_eval_per_class = sort_class_extract([val_loader2007, val_loader2012])


### Entraînement sur chaque classe.


In [ ]:
for i in tq.tqdm(range(len(classes))):
    classe = classes[i]
    print("Classe " + str(classe) + "...")
    agents_per_class[classe] = Agent(
        classe, alpha=0.2, num_episodes=15, load=False, arch=arch_type
    )
    agents_per_class[classe].train(datasets_per_class[classe])
    del agents_per_class[classe]
    torch.cuda.empty_cache()


In [ ]:
r = open("logs_over_epochs", "r")
d = r.readlines()
d = [i.strip() for i in d]

print(d[0])
print(d[1])
print(d[2])
print(d[3])
print(d[4])
print(d[5])
print(d[6])
print(d[7])
print(list(filter(None, d[1].split(" "))))


aps = {}
aps["0.1"] = []
aps["0.2"] = []
aps["0.3"] = []
aps["0.4"] = []
aps["0.5"] = []

recalls = {}
recalls["0.1"] = []
recalls["0.2"] = []
recalls["0.3"] = []
recalls["0.4"] = []
recalls["0.5"] = []


for i in range(1, 130, 3):  # (17):
    a = list(filter(None, d[i + 1].split(" ")))
    b = list(filter(None, d[i + 2].split(" ")))
    c = list(filter(None, d[i + 3].split(" ")))
    print("a : " + str(a))
    print("b : " + str(b))
    print("c : " + str(c))
    aps["0.1"].append(float(a[1]))
    aps["0.2"].append(float(a[2]))
    aps["0.3"].append(float(a[3]))
    aps["0.4"].append(float(a[4]))
    aps["0.5"].append(float(a[5]))

    recalls["0.1"].append(float(b[1]))
    recalls["0.2"].append(float(b[2]))
    recalls["0.3"].append(float(b[3]))
    recalls["0.4"].append(float(b[4]))
    recalls["0.5"].append(float(b[5]))

ar = []


ar = np.array(ar)
print(len(plt.plot(ar)))
labels = [
    "Threshold 0.1",
    "Threshold 0.2",
    "Threshold 0.3",
    "Threshold 0.4",
    "Threshold 0.5",
]
colors = ["r", "g", "b", "brown", "purple"]
c = 0
for key in aps:
    # sns.plot(aps[key], label=labels[c])
    sns.scatter(aps[key], label=labels[c], color=colors[c])
    c += 1
plt.xlabel("Episode")
plt.ylabel("AP")
plt.legend()
plt.show()

In [ ]:
with open('classes_results.pickle', 'rb') as handle:
    resss = pickle.load(handle)

# Define the class names
classes = [
    'cat', 'bird', 'motorbike', 'diningtable', 'train', 'tvmonitor', 
    'bus', 'horse', 'car', 'pottedplant', 'person', 'chair', 'boat', 
    'bottle', 'bicycle', 'dog', 'aeroplane', 'cow', 'sheep', 'sofa'
]

# Define the number of classes (sample count per class)
n_classes = {
    'cat': 648, 'bird': 553, 'motorbike': 304, 'diningtable': 188, 'train': 369, 
    'tvmonitor': 290, 'bus': 268, 'horse': 310, 'car': 659, 'pottedplant': 202, 
    'person': 1301, 'chair': 379, 'boat': 289, 'bottle': 258, 'bicycle': 303, 
    'dog': 750, 'aeroplane': 432, 'cow': 210, 'sheep': 208, 'sofa': 297
}

# Define the paper results
paper_results = {
    'cat': 55.9, 'bird': 38.4, 'motorbike': 55.9, 'diningtable': 46.3, 'train': 54.7, 
    'tvmonitor': 51.4, 'bus': 56.5, 'horse': 56.9, 'car': 58.8, 'pottedplant': 21.1, 
    'person': 45.7, 'chair': 21.4, 'boat': 36.5, 'bottle': 21.4, 'bicycle': 61.9, 
    'dog': 54.2, 'aeroplane': 55.5, 'cow': 40.4, 'sheep': 47.1, 'sofa': 41.5
}

# Extract AP values from `resss`
Y_obtenu = []
Y_paper = []
res_class = []

for cls in classes:
    # Get AP from the loaded pickle results safely
    ap_value = resss.get(cls, {}).get(0.4, {}).get('ap', 0)  # Default to 0 if key not found
    Y_obtenu.append(ap_value)
    Y_paper.append(paper_results[cls])
    res_class.append(cls)  # Duplicate class for grouping
    res_class.append(cls)

# Create DataFrame
df = pd.DataFrame({
    'Origin': ['Obtenu'] * len(Y_obtenu) + ['Papier'] * len(Y_paper),
    'AP': Y_obtenu + Y_paper,
    'Classe': res_class
})

# Plot
fig, ax = plt.subplots(figsize=(20, 10))  # Adjust size for better readability
ax.set_xlabel('AP', fontsize=20)
ax.set_ylabel('Classe', fontsize=20)
ax.set_xlim([0, 100])

sns.barplot(x='AP', y='Classe', data=df, hue='Origin', palette="magma")

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc='upper left', fontsize=15)
plt.tight_layout()

# Save & show plot
# plt.savefig('media/comparaison_classes.png', transparent=True)
plt.show()

In [ ]:
classe = "dog"
index = random.choice(list(datasets_per_class[classe].keys()))
agent = Agent(classe, alpha=0.2, num_episodes=25, load=True, arch=arch_type)

for i in range(20):
    index = random.choice(list(datasets_per_class[classe].keys()))
    image, gt_boxes = extract(index, datasets_per_class[classe])
    agent.predict_image(image, plot=True)

In [ ]:
classe = random.choice(classes)
index = random.choice(list(datasets_per_class[classe].keys()))
agent = Agent(classe, alpha=0.2, num_episodes=25, load=True, arch=arch_type)

for i in range(20):
    index = random.choice(list(datasets_per_class[classe].keys()))
    image, gt_boxes = extract(index, datasets_per_class[classe])
    agent.predict_image(image, plot=True)